In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display, Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

import datetime
import os

import joblib

In [2]:
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path)
  return model

In [3]:
IMG_SIZE = 264
def process_image(img_path):
  """
  Takes an image file path and turns it into a Tensor.
  """
  image = tf.io.read_file(img_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

In [4]:
def create_df(image, target):
    image = process_image(image)
    return image, target

In [5]:
df_train = pd.read_csv('data/train.csv')
filenames = ['data/jpeg/train/' + fname + '.jpg' for fname in df_train.image_name]

targets= df_train.target

X = filenames
y = pd.get_dummies(targets)

NUM_IMAGES = len(X)
BATCH_SIZE = 32


x_train, x_val, y_train, y_val = train_test_split(X[:NUM_IMAGES], y[:NUM_IMAGES], test_size = 0.2, random_state = 42)


train_data = tf.data.Dataset.from_tensor_slices((tf.constant(x_train), tf.constant(y_train)))
train_data = train_data.map(create_df).batch(BATCH_SIZE).shuffle(buffer_size=len(x_train))

val_data = tf.data.Dataset.from_tensor_slices((tf.constant(x_val), tf.constant(y_val)))
val_data = val_data.map(create_df).batch(BATCH_SIZE)

In [10]:
model = load_model('models/20200703152200-Images 0_30000_CNN.h5')

Loading saved model from: models/20200703152200-Images 0_30000_CNN.h5


In [12]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           multiple                  4864      
_________________________________________________________________
max_pooling2d_12 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_13 (Conv2D)           multiple                  204928    
_________________________________________________________________
max_pooling2d_13 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_14 (Conv2D)           multiple                  819456    
_________________________________________________________________
max_pooling2d_14 (MaxPooling multiple                  0         
_________________________________________________________________
flatten_4 (Flatten)          multiple                 

In [27]:
# Evaluate the loaded model
model.evaluate(val_data, verbose = 1)

AttributeError: 'RandomForestClassifier' object has no attribute 'evaluate'

In [28]:
df_test = pd.read_csv('data/test.csv')
filenames_test = ['data/jpeg/test/' + fname + '.jpg' for fname in df_test.image_name]
x_test = filenames_test

In [29]:
def create_df_test(image):
    image = process_image(image)
    return image

In [30]:
test_data = tf.data.Dataset.from_tensor_slices(tf.constant(x_test))
test_data = test_data.map(create_df_test).batch(BATCH_SIZE)

In [16]:
pred_test=model.predict(test_data, verbose=1)

NameError: name 'model' is not defined

In [17]:
test_data

<BatchDataset shapes: (None, 264, 264, 3), types: tf.float32>

In [18]:
image_name = df_test.image_name
target = []
for t in range(0,len(pred_test)):
    target.append(np.argmax(pred_test[t]))

NameError: name 'pred_test' is not defined

In [19]:
submission_df = pd.DataFrame(columns=['image_name']+['target'])

In [20]:
submission_df['image_name']= list(image_name)
submission_df['target'] = target

ValueError: Length of values does not match length of index

In [21]:
submission_df.to_csv('Submission/Submission3.csv', index = False)

In [22]:
tf.__version__

'2.2.0'